In [1]:
import tensorflow as tf

from tensorflow.keras.datasets import imdb

print(tf.version.VERSION)
print(tf.keras.__version__)

2.0.0
2.2.4-tf


In [2]:
def plot_accuracy(history_dict):
    
    acc = history_dict['binary_accuracy']
    
    epochs = range(1, len(acc) + 1)
    plt.clf() # clear figure
    acc_values = history_dict['binary_accuracy']
    val_acc_values = history_dict['val_binary_accuracy']

    plt.plot(epochs, acc_values, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc_values, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('accuracy')
    plt.legend()

    plt.show()

In [4]:
def test_hyperparameters(parameters):
    loss_function = parameters['loss_function']
    dense_layer_count = parameters['dense_layer_count']
    hidden_units = parameters['hidden_units']
    activation_function = parameters['activation_function']
    batch_size = parameters.get('batch_size') or 512
    lr = parameters.get('lr') or 0.001
    
    model = models.Sequential()
    model.add(layers.Dense(hidden_units, activation=activation_function, input_shape=(10000,)))
    for _ in range(dense_layer_count - 1):        
        model.add(layers.Dense(hidden_units, activation=activation_function))
        
    model.add(layers.Dense(1, activation='sigmoid'))

    #model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    model.compile(optimizer=optimizers.RMSprop(lr=lr), 
                  loss=loss_function, metrics=[metrics.binary_accuracy])   
    
    history = model.fit(partial_x_train, partial_y_train,
                   epochs=10, batch_size=batch_size, validation_data=(x_val, y_val))
    
    history_dict = history.history
    results = model.evaluate(x_test, y_test, verbose=0)

    print(results)
    plot_accuracy(history_dict)

In [9]:
from tensorflow.keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

print(len(train_data), len(test_data))

8982 2246


In [ ]:
train_data[10]

In [11]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in
train_data[0]])

557056/550378 [==============================] - 0s 0us/step


In [14]:
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [20]:
from tensorflow.keras.utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

#one_hot_train_labels = to_categorical(train_labels)
#one_hot_test_labels = to_categorical(test_labels)
